IMPORTS

In [6]:
# imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle
from openpyxl.utils import get_column_letter
from urllib.parse import quote

-------------------------------------------------------------------------------

DE WEBSCRAPING

----------------------------------------------------------

In [ ]:
excel_datei = 'Excel Datei mit Suchbegriffen in erster Spalte.xlsx'  # Dateiname der Excel-Datei
df = pd.read_excel(excel_datei)  # DataFrame aus der Excel-Datei lesen
spalte_hersteller_ccp = 'Hersteller_CCP'  # Spaltenname für den Hersteller CCP
driver = webdriver.Chrome()  # Webdriver initialisieren (Chrome)
aktueller_index = 0  # Aktueller Index für die Schleife

# Schleife über die Zeilen des DataFrames
while aktueller_index < len(df):
    wert = df.at[aktueller_index, spalte_hersteller_ccp]  # Wert in der Spalte 'Hersteller_CCP' für den aktuellen Index
    if pd.isnull(wert):  # Überprüfen, ob der Wert null ist
        break  # Schleife beenden, wenn der Wert null ist
    url = f"https://de.wikipedia.org/wiki/{wert}"  # URL für die Wikipedia-Seite des Herstellers erstellen
    driver.get(url)  # URL im Webdriver öffnen
    time.sleep(0.1)  # Kurze Wartezeit

    try:
        infobox_element = driver.find_element(By.CLASS_NAME, 'infobox')  # Infobox-Element auf der Seite finden

        if infobox_element is not None:
            rows = infobox_element.find_elements(By.TAG_NAME, 'tr')  # Alle Tabellenzeilen in der Infobox finden
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, 'td')  # Alle Zellen in der Tabellenzeile finden
                if len(cells) >= 2:  # Überprüfen, ob es mindestens zwei Zellen gibt
                    spaltenname = cells[0].text.strip().rstrip(':')  # Spaltenname extrahieren
                    wert = cells[1].text.strip()  # Wert extrahieren

                    df.at[aktueller_index, spaltenname] = wert  # Wert in die entsprechende Spalte des DataFrames einfügen
    except:
        pass

    aktueller_index += 1  # Index für die nächste Iteration erhöhen
    time.sleep(0.1)  # Kurze Wartezeit

df.to_excel('Gewünschter Name.xlsx', index=False)  # DataFrame als Excel-Datei speichern (ohne Index)
driver.quit()  # Webdriver beenden


DE WEBSITELINKS

In [8]:
excel_file_path = 'Gewünschter Name / Neue Excel'  # Pfad zur Excel-Datei
sheet_name = 'Sheet1'  # Name des Arbeitsblatts
column_name = 'Website'  # Name der Spalte

# Excel-Arbeitsmappe und Arbeitsblatt laden
workbook = load_workbook(excel_file_path)
worksheet = workbook[sheet_name]

column_index = None
# Suche nach dem Index der Spalte mit dem angegebenen Namen
for col in worksheet.iter_cols(1, worksheet.max_column):
    if col[0].value == column_name:
        column_index = col[0].column
        break

if column_index:
    hyperlink_style = "Hyperlink"  # Verwendung des eingebauten Hyperlink-Stils

    # Durchlaufe die Zeilen ab der zweiten Zeile in der Spalte mit dem angegebenen Index
    for row in worksheet.iter_rows(min_row=2, min_col=column_index, max_col=column_index):
        cell = row[0]
        if cell.value:
            search_query = quote(cell.value)  # URL-codierte Version des Zellwerts
            search_link = f'https://{search_query}'  # Suchlink mit dem URL-codierten Wert
            worksheet[cell.coordinate].value = cell.value  # Zellwert beibehalten
            worksheet[cell.coordinate].hyperlink = search_link  # Hyperlink setzen
            worksheet[cell.coordinate].style = hyperlink_style  # Stil des Hyperlinks setzen

workbook.save(excel_file_path)  # Arbeitsmappe speichern



--------------------------------------------------------------------------------------------

EN WEBSCRAPING

--------------------------------------------------------------------------------------------------

In [ ]:
excel_datei = 'Excel mit Suchbegriffen in erster Spalte.xlsx'  # Pfad zur Excel-Datei
df = pd.read_excel(excel_datei)  # DataFrame aus der Excel-Datei erstellen
spalte_hersteller_ccp = 'Hersteller_CCP'  # Name der Spalte für Hersteller-CCP
driver = webdriver.Chrome()  # Webdriver für Chrome initialisieren
aktueller_index = 0  # Startindex für die Iteration

while aktueller_index < len(df):
    wert = df.at[aktueller_index, spalte_hersteller_ccp]  # Wert in der Spalte Hersteller-CCP für den aktuellen Index abrufen

    if pd.isnull(wert):  # Wenn der Wert Null (NaN) ist, die Schleife abbrechen
        break

    url = f"https://en.wikipedia.org/wiki/{wert}"  # URL für die Wikipedia-Seite mit dem Wert erstellen
    driver.get(url)  # Die Wikipedia-Seite im WebDriver öffnen
    time.sleep(0.1)  # Kurze Wartezeit

    try:
        infobox_element = driver.find_element(By.CLASS_NAME, 'infobox.vcard')  # Das Infobox-Element mit der Klasse 'infobox.vcard' finden
        labels = infobox_element.find_elements(By.CLASS_NAME, 'infobox-label')  # Labels innerhalb der Infobox finden
        data = infobox_element.find_elements(By.CLASS_NAME, 'infobox-data')  # Daten innerhalb der Infobox finden

        for label, datum in zip(labels, data):  # Schleife über die Labels und Daten
            spaltenname = label.text.strip()  # Text des Labels bereinigen und als Spaltenname verwenden
            zelleninhalt = datum.text.strip()  # Text der Daten bereinigen und als Zelleninhalt verwenden
            df.at[aktueller_index, spaltenname] = zelleninhalt  # Wert in den DataFrame eintragen

    except:
        pass

    aktueller_index += 1  # Den Index für die nächste Iteration erhöhen
    time.sleep(0.1)  # Kurze Wartezeit

df.to_excel('Gewünschter Name.xlsx', index=False)  # DataFrame in eine Excel-Datei speichern
driver.quit()  # Den WebDriver schließen

EN WEBSITELINKS

In [10]:
excel_file_path = 'Gewünschter Name / Neue Excel'  # Pfad zur Excel-Datei
sheet_name = 'Sheet1'  # Name des Arbeitsblatts
column_name = 'Website'  # Name der Spalte für die Website-Links

workbook = load_workbook(excel_file_path)  # Excel-Datei laden
worksheet = workbook[sheet_name]  # Arbeitsblatt auswählen

column_index = None
for col in worksheet.iter_cols(1, worksheet.max_column):  # Iteration über die Spalten des Arbeitsblatts
    if col[0].value == column_name:  # Überprüfen, ob der Spaltenname mit 'Website' übereinstimmt
        column_index = col[0].column  # Spaltenindex speichern
        break

if column_index:  # Wenn der Spaltenindex gefunden wurde
    hyperlink_style = "Hyperlink"  # Verwendung des integrierten Hyperlink-Stils

    for row in worksheet.iter_rows(min_row=2, min_col=column_index, max_col=column_index):
        # Iteration über die Zeilen ab der zweiten Zeile in der Website-Spalte
        cell = row[0]  # Zelle in der aktuellen Zeile und Spalte

        if cell.value:  # Wenn die Zelle einen Wert enthält
            search_query = quote(cell.value)  # URL-kodierten Suchbegriff erstellen
            search_link = f'https://{search_query}'  # Link für die Suche erstellen

            worksheet[cell.coordinate].value = cell.value  # Wert der Zelle beibehalten
            worksheet[cell.coordinate].hyperlink = search_link  # Hyperlink für die Zelle festlegen
            worksheet[cell.coordinate].style = hyperlink_style  # Stil der Zelle auf Hyperlink setzen

workbook.save(excel_file_path)  # Excel-Datei speichern

